# Notes from the *Python and Vectorization* video series

In [6]:
import time, numpy as np, math

## Video: Vectorization

Compare vectorized and non-vectorized operations:

In [4]:
# Initialize two random arrays
a = np.random.rand(1000000)
b = np.random.rand(1000000)

# Test non-vectorized version 
c = 0
tic = time.time()
for i in range(1000000):
    c += a[i] * b[i]
toc = time.time()

print(c)
print("Non-vectorized version: " + str(1000*(toc-tic)) + "ms\n")

# Test vectorization version 
tic = time.time()
c = np.dot(a,b)
toc = time.time()

print(c)
print("Vectorized version: " + str(1000*(toc-tic)) + "ms")

250055.0846118524
Non-vectorized version: 890.1820182800293ms

250055.0846118598
Vectorized version: 3.3097267150878906ms


The explicit for loop and non-vectorized version took about 890.18ms. The vectorized version took approx. 3.31ms. 

Vectorization can significantly speed up your code. Try to avoid explicit for loops whenever possible. 

## Video: More Vectorization Examples

Say you need to apply the exponential operation on every element of a matrix/vector.

In [29]:
# Initialize random array 
v = np.random.rand(1000000)
n = len(v)

# Vectorized version
u = np.zeros((n, 1))
tic = time.time()
for i in range(n):
    u[i] = math.exp(v[i])
toc = time.time()
print("Non-vectorized version: " + str(1000*(toc-tic)) + "ms")

# Non-vectorized version
tic = time.time()    
u = np.exp(v)
toc = time.time()
print("Vectorized version: " + str(1000*(toc-tic)) + "ms")

Non-vectorized version: 1229.051113128662ms
Vectorized version: 14.938116073608398ms


Again, the vectorized version is much faster.

The NumPy library has more built-in functions to compute vector operations, such as:
- `np.log()`
- `np.abs()`
- `np.maximum()`

## Video: Vectorizing Logistic Regression

**How to perform logistic regression without explicit for loops:**

In our logistic regression gradient descent example earlier this week, we wrote 2 explicit for loops, the first of which propagates forward through the neural net. It iterates over m training examples to calculate the prediction a (or ŷ) of that training example with the help of a subfunction z. 

Original, highly inefficient non-vectorized logistic regression implementation: 

```
J = 0
dw1 = 0
dw2 = 0
db = 0

for i in range(1, m):
    z = (wT * x) + b
    a = σ(z)
    J += -[(y * log a) + ((1 - y) * log(1 - a))]
    dz = a - y
    dw1 += x1 * dz
    dw2 += x2 * dz
    db += dz

J = J/m
dw1 = dw1/m
dw2 = dw2/m
db = db/m
```

However, instead of doing the first for loop, we can run 

```
Z = np.dot(wT, x) + b
A = σ(Z)
```

to compute all of the z's and a's at the same time. The output of Z and A are 1xm vectors containing one z or a for each example. 

*Note: The T in wT denotes its transpose. It is supposed to be shown as a superscript.*

## Video: Vectorizing Logistic Regression's Gradient Computation

**How to use vectorization to also perform gradient descent:**

Continuing from the last video, with dw1 and dw2, we see the second for loop, which is part of the backward propagation through the neural net. It iterates over n features to get the derivatives (or slopes) of each feature of that training example. Here n = 2, so we have dw1 and dw2. If n = 5, we would have dw1 through dw5. 

To vectorize this for loop, we replace it with 

```
dw += x * dz
```

Putting together our vectorized for loops from the last video and this video, the updated full implementation is

```
Z = np.dot(wT, x) + b
A = σ(Z)
dZ = A - Y
dw = 1/m * dzT
db = 1/m * np.sum(dZ)
```

We've now done front propagation and back propagation, computing the predictions and derivatives on all m training examples without using a full loop.

And after we update the weight and loss variables, 

```
w = w - (α * dw)
b = b - (α * db)
```

we've implemented a single iteration of gradient descent for logistic regression. 

*Note: α, or alpha, is the learning rate.*

## Video: Broadcasting in Python

Broadcasting is a technique that Python and numpy use to allow you to make certain parts of your code much more efficient, e.g. removing for loops. 

Take this matrix:

![broadcasting_in_python.png](./broadcasting_in_python.png?raw=true)

Without explicit looping, we can add up the columns to find the total number of calories for apples, beef, eggs, and potatoes.

In [10]:
# Intialize the matrix
A = np.array([[56.0, 0.0, 4.4, 68.0],
              [1.2, 104.0, 52.0, 8.0],
              [1.8, 135.0, 99.0, 0.9]])
print(A)

[[ 56.    0.    4.4  68. ]
 [  1.2 104.   52.    8. ]
 [  1.8 135.   99.    0.9]]


In [12]:
# Sum each column
cal = A.sum(axis=0) 
print(cal)

[ 59.  239.  155.4  76.9]


The total number of calories for apples, beef, eggs, potatoes is 59, 239, 155.4, and 76.9, respectively.

*Note: With axis = 0 we are specifying we want to operate on columns, whereas axis = 1 is rows.*

In [15]:
# Calculate the percentage of carb, protein, fat in each food item
percentage = 100 * A/cal
print(percentage)

[[94.91525424  0.          2.83140283 88.42652796]
 [ 2.03389831 43.51464435 33.46203346 10.40312094]
 [ 3.05084746 56.48535565 63.70656371  1.17035111]]


94.9% of the calories in apples are from carbs, and so on.

We had a 3x4 matrix (`A`) and we divided it by a 1x4 matrix (`cal`). How is this possible? Through broadcasting.

For example, if you take a 4x1 vector, such as `[1, 2, 3, 4]`, and add it to a number, say 100, what Python will do is take this number and auto-expand it into a 4x1 vector as well, so it becomes `[100, 100, 100, 100]`, and then the sum is `[101, 102, 103, 104]`.

As another example, if we have (m, n) and (1, n) matrices

$$
\left(\begin{array}{@{}ccc@{}}
                    1 & 2 & 3 \\
                    4 & 5 & 6\end{array}\right) + \left(\begin{array}{@{}ccc@{}}
                    100 & 200 & 300 \end{array}\right)$$
                    
the 1xn matrix gets expanded into (m, n) dimensions, and the sum is 

$$
\left(\begin{array}{@{}ccc@{}}
                    101 & 202 & 303 \\
                    104 & 205 & 306\end{array}\right)$$
                   
We did broadcasting earlier with the constant b in logistic regression, using a division operation.

Broadcasting documentation: https://docs.scipy.org/doc/numpy-1.15.0/user/basics.broadcasting.html

## Video: A note on python/numpy vectors

Tips and tricks to avoid strange bugs:

In [41]:
# Create 5 random Gaussian variables stored in an array
a = np.random.randn(5)
print(a)

[0.15568529 0.82572763 0.57840157 0.65935008 0.1244858 ]


In [42]:
print(a.shape)
assert a.shape == (5,1), "Not a vector" 

(5,)


AssertionError: Not a vector

This shape indicates that a is a rank 1 array in Python and is neither a matrix nor a row vector. This has some slightly non-intuitive effects. 

In [21]:
# Print a-transpose
print(a.T)

[-1.380593   -0.6866715  -1.89812336 -1.43680426  0.2762907 ]


The transpose looks the same as a. 

In [22]:
# Print the inner product between a and a-transpose
print(np.dot(a, a.T))

8.121170129144351


We might think that a times a tranpose would give us a matrix. But we get back a number. 

When coding neural networks, don't use data structures that are not vectors or matrices. They will not behave like we expect, because they simply are not vectors or matrices.

Instead, tell Python we want a column vector:

In [38]:
# Create 5 random Gaussian variables stored in a column vector
a = np.random.randn(5,1)
print(a)

[[-0.8660183 ]
 [-1.48532738]
 [-2.25084207]
 [ 0.95650147]
 [ 0.87325993]]


In [40]:
assert a.shape == (5,1), "Not a vector" # Does not throw an AssertionError because it is a vector now

In [25]:
# Print a-transpose
print(a.T)

[[-1.22346984  0.03112995  0.35188154  0.62692364  0.66043587]]


Note that there are two square brackets when creating a transpose of a 1x5 matrix. 

If we do end up with a rank 1 array, we can reshape it. 

In [43]:
# Reshape the rank 1 array into a vector
a = np.random.randn(5)
print(a)
a = a.reshape(5, 1)
print(a)

[-0.9396036  -0.71911714 -1.17814407  1.14204755  0.23493005]
[[-0.9396036 ]
 [-0.71911714]
 [-1.17814407]
 [ 1.14204755]
 [ 0.23493005]]
